In [ ]:
readFiles = False
%run ./init.ipynb

In [ ]:
def in_notebook():
    """
    Returns ``True`` if the module is running in IPython kernel,
    ``False`` if in IPython shell or other Python shell.
    """
    return 'ipykernel' in sys.modules

def ipython_info():
    ip = False
    if 'ipykernel' in sys.modules:
        ip = 'notebook'
    elif 'IPython' in sys.modules:
        ip = 'terminal'
    return ip

In [ ]:
def renameFileIfExist(fileName):
    if os.path.exists(fileName):
        os.rename(fileName,'old_' + fileName)

In [ ]:
def saveDictionary(dataDictionary, filename, path):
    if not os.path.exists(path):
        os.makedirs(path)

    renameFileIfExist(fileName)
    fileAndPath = path + filename
    print(fileAndPath)
    dataDictionary.to_csv(fileAndPath)

In [ ]:
%%time

startRange = 1
stopRange = 100001
modulo = 5000

print(ipython_info()) 

#logging.basicConfig(level=logging.DEBUG)

if (ipython_info() == 'terminal'):
    if len(sys.argv) > 0:
        print('in sys if statement')
        startRange = int(sys.argv[1])
        stopRange = int(sys.argv[2])
        modulo = int(sys.argv[3])
        print('start range is ' + str(startRange))
        print('stop range is ' + str(stopRange))
        print('modulo is ' + str(modulo))


baseURL='https://aggregator.api.niftys.com/v1/metadata/contracts/'
#contractURL='ckw5j7pqf0011g4731pvaj6gm/'
#contractURL='matrix-red-contract/'

retrievedColumns=[]

for contractURL in contractURLs:
    rowAttributes = {}
    listing = {}
    blankOutputList = []
    retrievedColumns.clear()
    refStartx = startRange

    print('startRange = ' + str(startRange) + ' and stopRange = ' + str(stopRange))

    for avatarNumber in range(startRange, stopRange):

        s = requests.Session()
        retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
        s.mount('https://', HTTPAdapter(max_retries=retries))

        lineItemJson = s.get(baseURL + contractURL + str(avatarNumber)).text
        
        #lineItemJson = requests.get(baseURL + contractURL + str(avatarNumber)).text
        
        metadata = json.loads(lineItemJson)
        output_list=[]
        
        for k,v in metadata.items():
            if type(v) is list:
                if (len(retrievedColumns) == 0):
                    coulumnsNameAndLocation = './data/' + contractURL + 'retrievedColumnsList.txt'
                    if os.path.exists(coulumnsNameAndLocation):
                        #print('path exists')
                        with open(coulumnsNameAndLocation, 'r') as filehandle:
                            retrievedColumns = json.load(filehandle)
                    else:
                        #print('in else')
                        for i in v:
                            attributeNameList=list(i.values())
                            retrievedColumns.append(attributeNameList[0]) # name or key of the attribute
                            blankOutputList.append('')  # fill empty string to match the same size total
                         # open output file for writing
                        if (len(retrievedColumns) > 0):
                            #print('length of retrievedColumns is ' + str(len(retrievedColumns)))
                            
                            try:
                                with open(coulumnsNameAndLocation, 'w') as filehandle:
                                    json.dump(retrievedColumns, filehandle)
                            except:
                                print('made it to except')
                for attributes in v:
                    valueList=list(attributes.values())
                    cleanTextAttribute = valueList[0]
                    if (len(valueList[1]) > 0):
                        cleanTextValue = valueList[1]
                    else:
                        cleanTextValue = ''
                    rowAttributes[cleanTextAttribute] = cleanTextValue
                output_list = [(rowAttributes[key]) for key in (retrievedColumns) if key in rowAttributes]
        if len(output_list) == 0:
            listing[str(avatarNumber)] = blankOutputList.copy()
        else:
            listing[str(avatarNumber)] = output_list.copy()

        output_list.clear()
        clear_output(wait=True)
        print(contractURL, avatarNumber)
        if(avatarNumber % modulo == 0):
            if refStartx == 1:
                refStartx = 0
            data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=retrievedColumns)  #pd.DataFrame(listing, orient='index')
            fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(avatarNumber) + '.csv'
            saveDictionary(data_dictionary, fileName, './data/' + contractURL )
            refStartx = avatarNumber
            listing.clear()

In [ ]:
#fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv'
#renameFileIfExist(fileName)
#data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
#data_dictionary.to_csv(fileName)

In [ ]:
#for count in range(0, len(manualColumns)):
#    print(count)
#    print(data_dictionary[manualColumns[count]].value_counts())
#    data_dictionary[manualColumns[count]].value_counts(normalize=True)
#    print('---------------------------------')

In [ ]:
print('done')